In [ ]:
import pandas as pd
import time
from getpass import getpass
import os
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
from random import randint
import googlemaps

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/59.0'}

In [ ]:
page = requests.get("https://untappd.com/brewery/top_rated?country_id=130",headers = headers)
url= "https://untappd.com/brewery/top_rated?country_id="

In [ ]:
# get all <options> in a list
soup = BeautifulSoup(page.content, 'html.parser')
options = soup.find_all("option")

In [ ]:
# for each element in that list, pull out the "value" attribute
values = [o.get("value") for o in options]
del values[:8] #delete first row 'all' from list

In [ ]:
 for row in values:
        time.sleep(randint(5,10)) # wait between 5 to 10 seconds, not to hammer the servers with requests
        page1 = requests.get(url+str(row) ,  headers = headers)
        soup1 = BeautifulSoup(page1.content, 'html.parser')
        sf=soup1.find('p', class_='no-activity')
        
        if sf: #if you find the class no-activity, it means that the page is empty
            print(row,"No breweries listed")
            
        else:
            print(row,"printing... hopefully!")
            all_names=[] #all brewery names in a list
            names = soup1.find_all("p",  class_="name")
            for p in names: 
                names2 = p.text #cycle through names and drop all tags etc.
                all_names.append(names2) #add this new text only value to 'allnames'
            
            all_country=[]   #getting the country name for the list of breweries
            country = soup1.find_all("p",  class_="style")
            for p in country: 
                country2 = p.text #cycle through names and drop all tags etc.
                all_country.append(country2) #add this new text only value to 'allnames'
                
            all_nbeers=[]#number of beers listed
            nbeers = soup1.find_all("p",  class_="abv")
            for p in nbeers: 
                nbeers2 = p.text.replace('Beers','') #cycle through names and drop all "Beers"
                all_nbeers.append(nbeers2) #add this new text only value to 'allnames'
                
            all_nreviews=[] #number of reviewers
            nreviews = soup1.find_all("p",  class_="ibu")
            for p in nreviews: 
                nreviews2 = p.text.replace('Ratings','') #cycle through names and drop all tags etc.
                all_nreviews.append(nreviews2) #add this new text only value to 'allnames'
                
            all_rating=[]
            rating = soup1.find_all("p",  class_="rating")
            for p in rating: 
                rating2 = p.text.replace('(','').replace(')','') #cycle through names and drop all ( )
                all_rating.append(rating2) #add this new text only value to 'allnames'
                
            all_locations=[]
            all_styles=[]
            all_links=[] # list of links in order to get location we need to get into that webpage
            img_links=[] # list of images links
            
            prova=soup1.find_all('a', class_='label', href=True)
            img=soup1.find_all('img',src=True)
            #get all images url
            for image in img:
            
                img_links.append(image['src'])
            
            img_links= img_links[:len(img_links)-20] #delete last 20 rows from images list because they contain non related pics
            # get all info from beers page
            for p in prova: 
                time.sleep(randint(6,10))  
                all_links.append('https://untappd.com'+p['href'])
                
                brewery_detail = requests.get('https://untappd.com'+p['href']+'/beer', headers = headers)
                soup3=BeautifulSoup(brewery_detail.content, 'html.parser')
                
                try:
                    location=soup3.find('p', class_='brewery').text
                
                    all_locations.append(location) #append location as written on the website

                    style=soup3.find('p', class_='style').text
                    all_styles.append(style) #brewery style as on the website 
                    
                    #get beer id    
                    beer_id = [d.get("data-bid") for d in soup3.find_all("div", {"class": "beer-item"})]

                    #get beer names
                    beers_name=soup3.find_all('p', class_='name')
                    br_beers_name = [] #create empty list

                    for p in beers_name: 
                        beers_names2 = p.text #cycle through names and drop all tags etc.
                        br_beers_name.append(beers_names2)#add this new text only value to 'allnames'

                    #get full description going through each beer id

                    beer_description=[]

                    for d in beer_id:
                        b_descr=soup3.find('p', class_='desc desc-full-'+str(d))
                        beer_description.append(b_descr.text)

                    #get brewery name and repeat it for the number of beers listed
                    brewery_n=soup3.find('h1').text
                    brewery_name=[]
                    brewery_name=np.repeat(brewery_n,len(beer_id))

                    beer_style = [] #create empty list for beer style
                    b_style=soup3.find_all('p', class_='style')

                    for p in b_style: 
                        b_style2 = p.text #cycle through names and drop all tags etc.
                        beer_style.append(b_style2)#add this new text only value to 'allnames'

                    del beer_style[0] #delete first entry as it is the style of the brewery not the beer

                        #get beers ABV
                    ABVs=[]

                    b_abv=soup3.find_all('p',class_='abv')

                    for p in b_abv:
                        b_abv=p.text.replace('%','').replace('ABV','')
                        #float(b_abv)
                        ABVs.append(b_abv)

                    #get beers International Bitterness Unit
                    IBUs=[]
                    b_ibu=soup3.find_all('p',class_='ibu')

                    for p in b_ibu:
                        b_ibu=p.text.replace('IBU','')
                        IBUs.append(b_ibu)

                    #ratings
                    b_ratings=[]

                    b_r=soup3.find_all('p', class_='rating')

                    for p in b_r:
                        b_r=p.text.replace('(','').replace(')','')
                        b_ratings.append(b_r)

                    del b_ratings[0] #this is the rating of the brewery 

                    #raters
                    b_raters=[]

                    b_rtrs=soup3.find_all('p', class_='raters')

                    for p in b_rtrs:
                        b_rtrs=p.text.replace('Ratings','')
                        b_raters.append(b_rtrs)

                    del b_raters[0] #this is the raters of the brewery 

                    #date added
                    date_added=[]

                    date=soup3.find_all('p', class_='date')

                    for p in date:
                        date=p.text.replace('Added','')
                        date_added.append(date)

                    del date_added[0] #this is the date the brewery was added

                    df2 = pd.DataFrame(np.column_stack([beer_id,brewery_name,br_beers_name, beer_style, beer_description, ABVs, IBUs, b_raters, b_ratings, date_added]), 
                          columns=['Beer ID','Brewery', 'Beer name', 'Beer Style', 'Beer Description', 'ABV', 'IBU', 'No Raters', 'Rating', 'Date Added'])

                    df2.to_csv('All Beers.csv',mode='a', header=False, encoding='utf-8')
                except AttributeError:
                    all_locations.append("NA")
                    all_styles.append("NA") 
                    continue
   ################################         
            
            
            #create a table using the formatted lists from beautifulsoup delimited with commas
            table = {'Brewery': [all_names], 'Country': [all_country], 'No. Beers': [all_nbeers], 'No. Reviews': [all_nreviews], 'Rating': [all_rating],
                    'Untappd URL': [all_links], 'Image URL': [img_links], 'Style': [all_styles], 'Location': [all_locations] }
            #print(table)
            #create and populate a pandas dataframe from the table using numpy
            df1 = pd.DataFrame(np.column_stack([all_names, all_country, all_nbeers, all_nreviews, all_rating, all_links, img_links, all_styles, all_locations]), 
                  columns=['Brewery', 'Country', 'No. Beers', 'No. Reviews', 'Rating','Untappd URL','Image URL', 'Style', 'Location'])
        

            df1.to_csv('Breweries.csv', mode='a', header=False, encoding='utf-8') #keep appending onto csv

In [ ]:
gmaps_key = googlemaps.Client(key = 'YOUR PERSONAL KEY')

df1["Lat"] = None
df1["Lon"] = None

In [ ]:
for i in df1.index:
    time.sleep(1)
    geocode_result = gmaps_key.geocode(df1.iat[i,9])
    try:
        lat=geocode_result[0]["geometry"]["location"]["lat"]
        lon=geocode_result[0]["geometry"]["location"]["lng"]
        df1.at[i, "Lat"] = lat 
        df1.at[i, "Lon"] = lon
    except:
        lat = None
        lon = None

geocode_result = gmaps_key.geocode(df.iat[0,8])

In [ ]:
lat=geocode_result[0]["geometry"]["location"]["lat"]
lon=geocode_result[0]["geometry"]["location"]["lng"]
df.iat[0, df.columns.get_loc("Lat")] = lat
df.iat[0, df.columns.get_loc("Lon")] = lon